In [2]:
import pandas as pd
import requests
import os
import googleapiclient.discovery
import json

## Extract the Video Id

In [2]:
from googleapiclient.discovery import build

# Set up the YouTube API client (replace 'YOUR_API_KEY' with your API key)
api_key = 'YOUR_API_KEY'
youtube = build('youtube', 'v3', developerKey=api_key)

# Video URL
video_url = 'https://www.youtube.com/watch?v=MVYrJJNdrEg&t=793s'

# Extract the video ID from the URL
video_id = video_url.split('v=')[1]

print(f'Video ID: {video_id}')

Video ID: MVYrJJNdrEg&t=793s


## Extract the Channel Id

In [21]:
# Channel Username or Channel URL
channel_username = '@lexfridman'  # Or replace with the full channel URL
if 'channel/' in channel_username:
    channel_username = channel_username.split('channel/')[-1]

# Make an API request to get the channel ID by channel username or URL
request = youtube.channels().list(part='id', forUsername=channel_username)
response = request.execute()


#response
# Extract the channel ID
channel_id = response['etag']
print(f'Channel ID for {channel_username}: {channel_id}')

Channel ID for @lexfridman: RuuXzTIr0OoDqI4S0RU6n4FqKEM


In [295]:
def data_extraction(datafile, query_num):
    row_data = []
    
    for data in datafile:
        df_dict = {}
        df_dict["published_At"] = data["snippet"]["topLevelComment"]["snippet"]["publishedAt"]
        df_dict["updated_At"] = data["snippet"]["topLevelComment"]["snippet"]["updatedAt"]
        df_dict["author_DisplayName"] = data["snippet"]["topLevelComment"]["snippet"]["authorDisplayName"]
        df_dict["text_Original"] = data["snippet"]["topLevelComment"]["snippet"]["textOriginal"]
        df_dict["text_Display"] = data["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
        df_dict["viewer_Rating"] = data["snippet"]["topLevelComment"]["snippet"]["viewerRating"]
        df_dict["like_Count"] = data["snippet"]["topLevelComment"]["snippet"]["likeCount"]
        df_dict["author_Channel_Id"] = data["snippet"]["topLevelComment"]["snippet"]["authorChannelId"]["value"]

        row_data.append(df_dict)
        

    df = pd.DataFrame(row_data)
    df.to_csv(f"comment_data{query_num}.csv")

    return df

## Comment Scraper

In [41]:
url = "https://www.googleapis.com/youtube/v3/comments"

In [296]:
next_page_token = None
total_comments = 0
comment_id_set = set()
query_count = -1

page_dicts = []

while True:
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = api_key

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.commentThreads().list(
        part="snippet",
        videoId="MVYrJJNdrEg",
        maxResults=100,
        pageToken=next_page_token
        
    )

    
    responses = request.execute()

       
    print("Data Returned Succesfully!")

    idx_num = 0
    with open(f"raw_scraped_data{idx_num}.json", "w") as json_file:
        json.dump(responses, json_file)
        json_file.close()

    with open(f"raw_scraped_data{idx_num}.json", "r") as json_file:
        data_load = json.load(json_file)
        data = data_load["items"]

    query_count += 1
    data_dict_list = data_extraction(data, query_num=query_count)          ####### Returns a List of dictionaries for the current page
    
    page_dicts.append(data_dict_list)
         

    next_page_token = responses.get("nextPageToken")
    
    # Exit Code after last page
    if not next_page_token:
        break
       

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
4


## Test Code:

In [166]:
# data_extraction(data)

,published_At,updated_At,author_DisplayName,text_Original,text_Display,viewer_Rating,like_Count,author_Channel_Id
0,2023-09-29T01:58:46Z,2023-09-29T01:58:46Z,Justin Rogo,Bruh,Bruh,none,0,UChytvY-VQw13Z7JZKLPuMMg
1,2023-09-29T01:58:40Z,2023-09-29T01:58:40Z,b b,This episode jumped the shark.,This episode jumped the shark.,none,0,UCLvaDjlj8bMmKbXH6kFV8lw
2,2023-09-29T01:58:39Z,2023-09-29T01:58:39Z,marna lam,Good job on interviewing a person who interfer...,Good job on interviewing a person who interfer...,none,0,UCbXWEoVEa6j8qos1qAUxYEQ
3,2023-09-29T01:58:26Z,2023-09-29T01:58:26Z,fulltrottle911,"I feel like Mark was about to say "" is that FR...",I feel like Mark was about to say &quot; is th...,none,0,UCW0twMnkf3I-ipTgT7PbiZw
4,2023-09-29T01:58:25Z,2023-09-29T01:58:25Z,Mathias M.,Super cool,Super cool,none,0,UCgrEfacAB1RwpifLfrglPag
5,2023-09-29T01:58:22Z,2023-09-29T01:58:22Z,whit meek,"It's not about bandwidth efficiency, I mean se...","It&#39;s not about bandwidth efficiency, I mea...",none,0,UCae5fGVy5JfhbtA2TpCpm_A
6,2023-09-29T01:58:19Z,2023-09-29T01:58:19Z,Momentum,"Holy shit, mind blown !","Holy shit, mind blown !",none,0,UCuzdGLAnBd5dKHd3i88OdoQ
7,2023-09-29T01:57:33Z,2023-09-29T01:57:33Z,Entropicvibe,Cyberpunk 2023,Cyberpunk 2023,none,0,UCDtT6K5FRkWKMxHAiaHN3qA
8,2023-09-29T01:57:29Z,2023-09-29T01:57:29Z,peruna soffa,"LOL. Skype, Facetime etc. haves existed, what,...","LOL. Skype, Facetime etc. haves existed, what,...",none,0,UCT8zYVsf5VIgN7dpMx-bDCA
9,2023-09-29T01:57:14Z,2023-09-29T01:57:14Z,Zane Roberts,Wow so crazy u made a face of a person that lo...,Wow so crazy u made a face of a person that lo...,none,0,UCk0sqfQsTLJjvtNETXKfYUA


## First Row Extraction 

In [134]:
data[0]["snippet"]["topLevelComment"]["snippet"] #["channelId"]

{'channelId': 'UCSHZKyawb77ixDdsGog4iWA',
 'videoId': 'MVYrJJNdrEg',
 'textDisplay': 'Bruh',
 'textOriginal': 'Bruh',
 'authorDisplayName': 'Justin Rogo',
 'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/APkrFKbEtENYEZcHB5ZDhWHUky_NJafYPKEGYuW1q-3-eQ=s48-c-k-c0x00ffffff-no-rj',
 'authorChannelUrl': 'http://www.youtube.com/channel/UChytvY-VQw13Z7JZKLPuMMg',
 'authorChannelId': {'value': 'UChytvY-VQw13Z7JZKLPuMMg'},
 'canRate': True,
 'viewerRating': 'none',
 'likeCount': 0,
 'publishedAt': '2023-09-29T01:58:46Z',
 'updatedAt': '2023-09-29T01:58:46Z'}

### Extract Comment's Unique Identifier
The "id" key contains this information
* data[0]["id"] for the first comment by Lex Fridman

In [287]:
data[0] #["channelId"]

{'kind': 'youtube#commentThread',
 'etag': 'iKkUjjENKxkQiIbhSN3uG27F0A8',
 'id': 'UgzyGIx1SVZH2efJXkt4AaABAg',
 'snippet': {'channelId': 'UCSHZKyawb77ixDdsGog4iWA',
  'videoId': 'MVYrJJNdrEg',
  'topLevelComment': {'kind': 'youtube#comment',
   'etag': 'iWDak37yBmUQVB84_Oxlj1qA1Tc',
   'id': 'UgzyGIx1SVZH2efJXkt4AaABAg',
   'snippet': {'channelId': 'UCSHZKyawb77ixDdsGog4iWA',
    'videoId': 'MVYrJJNdrEg',
    'textDisplay': 'Thank you for listening ❤ Here are the timestamps:\r<br>Transcript: <a href="https://lexfridman.com/mark-zuckerberg-3-transcript">https://lexfridman.com/mark-zuckerberg-3-transcript\r</a><br><a href="https://www.youtube.com/watch?v=MVYrJJNdrEg&amp;t=0m00s">0:00</a> - Introduction &amp; sponsor mentions:\r<br><a href="https://www.youtube.com/watch?v=MVYrJJNdrEg&amp;t=0m52s">0:52</a> - Metaverse\r<br><a href="https://www.youtube.com/watch?v=MVYrJJNdrEg&amp;t=15m27s">15:27</a> - Quest 3\r<br><a href="https://www.youtube.com/watch?v=MVYrJJNdrEg&amp;t=30m16s">30:16</a> 

In [291]:
data[1]

{'kind': 'youtube#commentThread',
 'etag': 'XKLSlYkPV0JsWwtgLNbBWxjGgsk',
 'id': 'UgyCQcoTNz3FkEkHv914AaABAg',
 'snippet': {'channelId': 'UCSHZKyawb77ixDdsGog4iWA',
  'videoId': 'MVYrJJNdrEg',
  'topLevelComment': {'kind': 'youtube#comment',
   'etag': 'WDxy0n8JYmndpXP_7B6LncwuBgA',
   'id': 'UgyCQcoTNz3FkEkHv914AaABAg',
   'snippet': {'channelId': 'UCSHZKyawb77ixDdsGog4iWA',
    'videoId': 'MVYrJJNdrEg',
    'textDisplay': 'They need to capture the bio frequencies of their bodies and transmit them too. Wait, that’s probably decades away. You can’t imitate the vibrations of every cell in the body.',
    'textOriginal': 'They need to capture the bio frequencies of their bodies and transmit them too. Wait, that’s probably decades away. You can’t imitate the vibrations of every cell in the body.',
    'authorDisplayName': 'Marnie Khaw',
    'authorProfileImageUrl': 'https://yt3.ggpht.com/wVcU-M-6nTM8u0Pr6rFuiuVBAFywFuTc7D-I2hNyjCR_KQwJwgc9d2_b9bljXE3Zyucc4w_AUg=s48-c-k-c0x00ffffff-no-rj',

In [3]:
testdf = pd.read_csv("comment_files/comment_data0.csv")
testdf.head()

,Unnamed: 0,published_At,updated_At,author_DisplayName,text_Original,text_Display,viewer_Rating,like_Count,author_Channel_Id
0,0,2023-09-28T17:05:17Z,2023-09-28T17:14:39Z,Lex Fridman,Thank you for listening ❤ Here are the timesta...,Thank you for listening ❤ Here are the timesta...,none,1012,UCSHZKyawb77ixDdsGog4iWA
1,1,2023-09-29T17:04:47Z,2023-09-29T17:04:47Z,Willem Asselbergs,Fantastic Lex. Thanks to Mark also for being s...,Fantastic Lex. Thanks to Mark also for being s...,none,0,UCKRDI-o1O8M3oG627Lx1CWQ
2,2,2023-09-29T17:04:37Z,2023-09-29T17:04:37Z,Vuk Djukic,congrats Lex - you became an NPC :D,congrats Lex - you became an NPC :D,none,0,UC1xlfdc3HASJ4PP_4nQxpug
3,3,2023-09-29T17:04:33Z,2023-09-29T17:04:33Z,Kreative school,💙💚❤️🧡💜,💙💚❤️🧡💜,none,0,UCueYF2Qmhda_xxzaH6Shqdw
4,4,2023-09-29T17:04:18Z,2023-09-29T17:04:18Z,dudedidyouseethat,After seeing this I'm convinced we are already...,After seeing this I&#39;m convinced we are alr...,none,0,UCpQSRC4T5rw364akt2HCmZw


In [4]:
testdf2 = pd.read_csv("comment_files/comment_data1.csv")
testdf2.head()

,Unnamed: 0,published_At,updated_At,author_DisplayName,text_Original,text_Display,viewer_Rating,like_Count,author_Channel_Id
0,0,2023-09-29T15:49:20Z,2023-09-29T15:49:20Z,Cyborg 1612,It’s giving Death Stranding,It’s giving Death Stranding,none,0,UCDE30H4UkYK400UNiDaZFow
1,1,2023-09-29T15:47:10Z,2023-09-29T15:47:10Z,BLSTC blender,nice try lex but we all know Mark Zuckerberg i...,nice try lex but we all know Mark Zuckerberg i...,none,0,UCxItz3Gu75RFRU446qQS3WA
2,2,2023-09-29T15:45:56Z,2023-09-29T15:45:56Z,Colin McIntosh,Black Mirror script writers watching this and ...,Black Mirror script writers watching this and ...,none,0,UCoXFkVqGMA62g5NEjRyQkMQ
3,3,2023-09-29T15:44:13Z,2023-09-29T15:44:13Z,Michael D. Anderson,Two observations:\n1. Lex probably isn't weari...,Two observations:<br>1. Lex probably isn&#39;t...,none,1,UC-lzRdnOByYmxZ4svtR2TIA
4,4,2023-09-29T15:43:59Z,2023-09-29T15:43:59Z,TheIronpusher,That's really cool Mark! The innovation looks ...,That&#39;s really cool Mark! The innovation lo...,none,0,UC-lWYv9_SkgOpwfzPFtj5cw
